Load all the packages needed

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Define the url to call wikipedia page

In [13]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
webpage = BeautifulSoup(url,'lxml')

search within the webpage where is the table and extract all the rows

In [17]:
table = webpage.find('table')
rows = table.find_all('td')

Loop to build a dataframe from the retrieved table

at the beginning of the loop there are 2 conditions which reflect the requirements of the assignement:
    
    1) fully "not assigned" rows should be disregarded
    2) when borough ok but neighboroughood "not assigned", then the neighboroughood should be equal to borough

In [18]:

code = []
borough = []
neighbourhood = []

for i in range(0, len(rows), 3):
    if (rows[i+1].text.strip() == 'Not assigned'and rows[i+2].text.strip() == 'Not assigned'):
        continue
    if (rows[i+1].text.strip() != 'Not assigned'and rows[i+2].text.strip() == 'Not assigned'):
        rows[i+2] = rows[i+1]
    postcode.append(rows[i].text.strip())
    borough.append(rows[i+1].text.strip())
    neighbourhood.append(rows[i+2].text.strip())
        
toronto_data_raw= pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
toronto_data_raw.columns = ['Postcode', 'Borough', 'Neighbourhood']

toronto_data = toronto_data_raw.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
toronto_data.columns = ['Postcode', 'Borough', 'Neighbourhood']

Finally use shape to confirm dimentions of the dataframe

In [20]:
toronto_data.shape

(103, 3)

display for visual ispection

In [22]:
toronto_data

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Retrieve latitute and longitude data from the csv file

In [31]:
Lat_Long_data = pd.read_csv('http://cocl.us/Geospatial_data')
Lat_Long_data .columns = ['Postcode', 'Latitude', 'Longitude']

use pandas "merge" to add columns longitude and latitude to corresponding rows

In [32]:
toronto_data_final = pd.merge(toronto_data, Lat_Long_data, on=['Postcode'], how='inner')

In [33]:
toronto_data_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


following packages are needed, please installed them if you didn't do it before

In [34]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Let's draw a Map of Toronto as it has been done for New York in the lab

In [35]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/Users/riccardo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Establish connection credential to call Foursquare API

In [37]:
CLIENT_ID = '10P0RJRLTA05UIMMWSZH0GQ4NBMPIA1HL0ISZYWLGHVLT0LB' # your Foursquare ID
CLIENT_SECRET = 'ZZ5F1IOT0WSTZUIWOR4OYC3IZEA12ZAEZC5C523BK1CCBWSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 10P0RJRLTA05UIMMWSZH0GQ4NBMPIA1HL0ISZYWLGHVLT0LB
CLIENT_SECRET:ZZ5F1IOT0WSTZUIWOR4OYC3IZEA12ZAEZC5C523BK1CCBWSR


After trying a few Neighourhoods, the first whose results where not empty was : "The Beaches". I continue with this

In [73]:
#The Beaches
toronto_data_final[toronto_data_final['Neighbourhood']=='The Beaches']

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [74]:
toronto_data_final.loc[37, 'Neighbourhood']

'The Beaches'

Find coordinates

In [80]:
neighborhood_latitude = toronto_data_final.loc[37, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data_final.loc[37, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data_final.loc[37, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [81]:
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, neighborhood_name, 500, 100)# type your answer here

url

'https://api.foursquare.com/v2/venues/explore?client_id=10P0RJRLTA05UIMMWSZH0GQ4NBMPIA1HL0ISZYWLGHVLT0LB&client_secret=ZZ5F1IOT0WSTZUIWOR4OYC3IZEA12ZAEZC5C523BK1CCBWSR&ll=43.67635739999999,-79.2930312&v=20180605&query=The Beaches&radius=500&limit=100'

Call Foursquare to get the results of the first 100 venues within 500m ( numbers have been chosen to replicate the lab as suggested by the assignment)

In [82]:
results = requests.get(url).json()
#results

In [84]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [85]:
import json
from pandas.io.json import json_normalize 

In [86]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Upper Beaches,Neighborhood,43.680563,-79.292869
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734


The conclusion i get from the resulting 2 venues list is that is interesting to notice that one of the top venues is a Helth Food Store. 
This might signal that people living in the area are more likely to have a healthy lifestyle especially when it comes to food.